In [ ]:
from RootInteractive.Tools.histoNDTools import *

# Random Data Frame generator

In [ ]:
npoints=10000
df = pd.DataFrame(np.random.random_sample(size=(npoints, 4)), columns=list('ABCD'))
df["noise"]=np.random.normal(0, 0.1, npoints)
df["csin"]=np.sin(6.28*df["C"])
df["valueOrig"]=df["A"]+np.exp(3*df["B"])*df["csin"]
df["value"]=df["valueOrig"]+df["noise"]
df.tail()


# Make Histogram

In [ ]:
histo=makeHistogram(df,"value:A:B:C:D:#A>0>>myhisto(50,-25,25,40,0,1,30,0,1,20,0,1,10,0,1)")


# Inputs

In [ ]:
slices=((0,50,1,0),(10,30,1,4),(10,24,5,3),(10,16,2,2),(0,10,3,0))   # start:stop:step:width
dimI=0                                                               # dimension of interest  

# The Code of the function

### Width modification

In [ ]:
binList = []                                # Prepare list of bin numbers which has same dimensions with original histogram
for el in histo["axes"]:                    #     so same slicing can be applied on the bin list
    binList.append(np.arange(len(el)-1))

    
widthList=[item[3] for item in slices]      # extract the list of widths from slicing tuple
localHist = histo["H"].copy()
localAxes = histo["axes"].copy()


for iDim, w in enumerate(widthList):          # get the original histogram and shifted histograms by +/- i,
    widthSliceList=[]                         #     where i is all integers upto width, sum all histograms 
    L=localHist.shape[iDim]
    
    iSlice=[slice(None)]*len(localHist.shape)
    iSlice[iDim]=slice(w,L-w)
    dummy=localHist[tuple(iSlice)] 
    
    for i in range(1,w+1):
        iSlice=[slice(None)]*len(localHist.shape)
        iSlice[iDim]=slice(w+i,L-w+i)
        dummy+=localHist[tuple(iSlice)]
        
        iSlice=[slice(None)]*len(localHist.shape)
        iSlice[iDim]=slice(w-i,L-w-i)
        dummy+=localHist[tuple(iSlice)]
        
    localHist = dummy
    binList[iDim] = binList[iDim][slice(w,L-w)]    # trim the edges
    localAxes[iDim] = localAxes[iDim][slice(w,L-w+1)]

print(widthList)
print(localHist.shape)
for a, b in zip(localAxes,binList):
    print(a.shape, b.shape)
print(binList)


### Slicing tuple

In [ ]:
newSliceList=[]
for iDim, iSlice in enumerate(slices):    # rearranging slice accourding to width
    jSlice=slice(iSlice[0] - iSlice[3], iSlice[1] - iSlice[3], iSlice[2])
        
    if jSlice.start < 0 or jSlice.stop >  localHist.shape[iDim]:
        raise ValueError("Range and width of {}th dimension is not compatible: {}".format(iDim,iSlice))
    
    newSliceList.append(jSlice)  
    
    
npSlice=tuple(newSliceList)


print(histo["H"].shape)
print(localHist.shape)
print(slices)
print(newSliceList)
print(npSlice)
print(npAxesSlice)
for a,b in zip(histo["axes"],localAxes):
    print(a.shape, b.shape)

### Determine the bin centers

In [ ]:
CenterList = []
for arr in localAxes:
    arri = arr[:-1] + (arr[1]-arr[0])/2
    CenterList.append(arri)
    
    
print(CenterList)

### aply the npslice

In [ ]:
histogram = localHist[npSlice]
localCenter = []
localNumbers = []

for i,(b,c) in enumerate(zip(binList,CenterList)):
    localNumbers.append(b[npSlice[i]])
    localCenter.append(c[npSlice[i]])

localCenter.pop(dimI)
localNumbers.pop(dimI)


print(localNumbers)
print(localCenter)

# Bin Number

In [ ]:
mesharrayNumbers = np.array(np.meshgrid(*localNumbers, indexing='ij'))

In [ ]:
binNumbers = []
for el in mesharrayNumbers:
    binNumbers.append(el.flatten())

# Bin Center

In [ ]:

mesharrayCenter = np.array(np.meshgrid(*localCenter, indexing='ij'))

In [ ]:
binCenters = []
for el in mesharrayCenter:
    binCenters.append(el.flatten())

In [ ]:
histogramMap={}
varNames=histo['varNames']
varNames.pop(dimI)

for iDim in range(len(varNames)):
    histogramMap[varNames[iDim] +"BinNumber"] = binNumbers[iDim]
    histogramMap[varNames[iDim] +"BinCenter"] = binCenters[iDim]
    print(len(binNumbers[iDim]))
    print(len(binCenters[iDim]))

means = np.ndarray.flatten(np.mean(histogram,axis=dimI))
rms = np.ndarray.flatten(np.sqrt(np.mean(histogram**2,axis=dimI)))
medians = np.ndarray.flatten(np.median(histogram,axis=dimI))
entries = np.ndarray.flatten(np.sum(histogram,axis=dimI))
print(means.shape)

histogramMap["means"] = means
histogramMap["rms"] = rms
histogramMap["medians"] = medians
histogramMap["entries"] = entries
out=pd.DataFrame(histogramMap)


In [ ]:
out.head